# Role Based Access Control

Role based access control provides or restricts access to the database resources based on the user's role within an organization.

In Milvus, we can create roles to which we can assign permissions.

We can assign one or more roles to an user, and this will grant all the permissions to the user from the assigned roles.

Before we begin with role based access control examples, we will have to enable user authentication in the settings.

To do that, we will have to download the configuration file, enable authentication in the configuration file and mount the file before we start the containers.

The download link for the config file is available in the official documentation: https://milvus.io/docs/configure-docker.md#Download-a-configuration-file

And in the downloaded file, milvus.yaml, we will have to set the authorization enabled parameter as true. And this is available under the security.
```
authorizationEnabled: true
```

Once the changes are saved in the file, open the docker compose file and add this configuration file as Mountable volume.

```
    volumes:
      - ${DOCKER_VOLUME_DIRECTORY:-.}/volumes/milvus:/var/lib/milvus
      - ${DOCKER_VOLUME_DIRECTORY:-.}/milvus.yaml:/milvus/config/milvus.yaml
```

Once these changes are made, make sure to restart the containers.

Once the authentication is enabled, we will have to connect using root user and the default password for the root user is milvus. We will have to provide these credentials while connecting to the Milvus server.

In [1]:
from pymilvus import Role, utility, connections, Collection
connections.connect(
  alias="default",
  host='localhost',
  port='19530',
    user='root',
    password='Milvus'
)

In [2]:
# Create a role
print(utility.list_roles(include_user_info=True, using="default"))

role_name = "test_role"
role = Role(role_name, using="default")
role.create()

print(utility.list_roles(include_user_info=True, using="default"))

RoleInfo groups:
- RoleItem: <role_name:admin>, <users:()>
- RoleItem: <role_name:public>, <users:()>
RoleInfo groups:
- RoleItem: <role_name:admin>, <users:()>
- RoleItem: <role_name:public>, <users:()>
- RoleItem: <role_name:test_role>, <users:()>


https://milvus.io/docs/users_and_roles.md#Users-and-Roles

In [3]:
# Grant Privileges to the role

role.grant("Collection", "*", "Search")
role.list_grants()

GrantInfo groups:
- GrantItem: <object:Collection>, <object_name:*>, <db_name:default>, <role_name:test_role>, <grantor_name:root>, <privilege:Search>

In [4]:
# Create a new user
utility.create_user('golden_clover', '123456', using="default")
print(utility.list_usernames(using="default"))

['golden_clover', 'root']


In [5]:
# Add role to user
print(utility.list_user('golden_clover', include_role_info=True, using="default"))

role.add_user('golden_clover')

print(utility.list_user('golden_clover', include_role_info=True, using="default"))

UserInfo groups:
- UserItem: <username:golden_clover>, <roles:()>
UserInfo groups:
- UserItem: <username:golden_clover>, <roles:('test_role',)>


In [6]:
# Revoke permissions

role.revoke("Collection","*","Search")
role.list_grants()

GrantInfo groups:

In [7]:
# Remove user from role

role.remove_user('golden_clover')

print(utility.list_user('golden_clover', include_role_info=True, using="default"))

UserInfo groups:
- UserItem: <username:golden_clover>, <roles:()>


In [8]:
# Drop a role

role.drop()
utility.list_roles(include_user_info=True, using="default")

RoleInfo groups:
- RoleItem: <role_name:admin>, <users:()>
- RoleItem: <role_name:public>, <users:()>

In [9]:
# Drop a user
utility.delete_user('golden_clover', using="default")
print(utility.list_usernames(using="default"))


['root']
